<a href="https://colab.research.google.com/github/joaopauloev/Data-Science/blob/master/Projeto_RAG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alunos:

- Vandelson Elias - vemf@cesar.school
- João Paulo Galindo Epifani Veloso - jpgev@cesar.school


## Construção de um RAG utilizando LangChain

Deverá ser desenvolvido um sistema RAG (Retrieval-Augmented Generation) utilizando a biblioteca LangChain para um ou mais documentos de sua escolha. Podendo ser arquivos PDF, de texto, páginas da web etc. O projeto será avaliado nos seguintes aspectos:

* Escolha do Documento
* Splitting do Documento
* Criação de Vector Store
* Retrieval
* Geração de Respostas

### Retrieval Augmented Generation (RAG)
Carregando Documentos da Web - Loading

In [8]:
!pip install langchain

In [3]:
!pip install --upgrade --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.2 MB/s eta 0:00:00


In [21]:
!pip install langchain_community

  Using cached langchain_community-0.3.14-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00


In [68]:
!pip install langchain_openai

  Using cached langchain_openai-0.3.1-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29


In [80]:
!pip install langchain faiss-cpu openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.3 MB/s eta 0:00:00


In [11]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [51]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

### * Escolha do documento

In [59]:
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader

uploaded = files.upload()

Saving Edital01.pdf to Edital01.pdf


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
file_path = (
    "/content/drive/MyDrive/rag/Edital01.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load_and_split()

docs[0]

Document(metadata={'source': '/content/drive/MyDrive/rag/Edital01.pdf', 'page': 0}, page_content='2 \n \nBANCO NACIONAL DE DESENVOLVIMENTO ECONÔMICO E SOCIAL - BNDES \nEDITAL Nº 01/2012 \nSELEÇÃO PÚBLICA \n \nO BANCO NACIONAL DE DESENVOLVIMENTO ECONÔMICO E SOCIAL - BNDES, através do Departamento de \nAdministração de Recursos Humanos, torna pública a realização de Sele ção Pública para a formação de Cadastros de \nReserva de Pessoal para o cargo de nível superior - Profissional Básico (formações de Administração, Análise de Sistemas \n- Desenvolvimento, Análise de Sistemas - Suporte, Arquitetura, Arquivologia, Biblioteconomi a, Comunicação Social , \nContabilidade, Direito, Economia, Engenharia e Psicologia) e para o cargo de nível médio - Técnico Administrativo, mediante \nas condições contidas neste Edital. \n1 - APRESENTAÇÃO \n1.1 - A Seleção Pública será regida por este Edital, seu s Anexos e eventuais retificações, promovida pela FUNDAÇÃO \nCESGRANRIO e realizada sob sua inteira r

In [61]:
print(docs[0].page_content[500:1000])

uivologia, Biblioteconomi a, Comunicação Social , 
Contabilidade, Direito, Economia, Engenharia e Psicologia) e para o cargo de nível médio - Técnico Administrativo, mediante 
as condições contidas neste Edital. 
1 - APRESENTAÇÃO 
1.1 - A Seleção Pública será regida por este Edital, seu s Anexos e eventuais retificações, promovida pela FUNDAÇÃO 
CESGRANRIO e realizada sob sua inteira responsabilidade, organização e controle. 
1.2 - A Seleção Pública se destina a selecionar candidatos para formaç


### * Splitting do Documento

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

179


In [63]:
print(all_splits[2].page_content)

Desenvolvimento, Análise de Sistemas - Suporte, Arquitetura, Arquivologia, Biblioteconomia , Comunicação Social , 
Contabilidade, Direito, Economia, Engenharia e Psicologia ) a serem admitidos , a disponibilidade para realizar viage ns a 
serviço, no país ou no exterior. 
1.4 - As relações de trabalho serão regidas pela Consolidação das Leis do Trabalho (CLT), pela legislação complementar e 
pelos regulamentos em vigor no Banco Nacional de Desenvolvimento Econômico e Social - BNDES na data de admissão do 
candidato contratado. 
1.5 - Os candidatos integrantes dos Cadastros de Reserva de Pessoal serão convocados, conforme necessidade e 
conveniência do BNDES e de acordo com a classificação obtida, por cargo ou cargo/formação, para comprovação  de 
requisitos exigidos, realização dos Exames Médicos (na forma especificada no subitem 10.1.3) e demais procedimentos pré-
admissionais, todos eliminatórios.


In [82]:
from langchain.schema import Document

all_splits = [
    Document(page_content="Texto do documento 1", metadata={"source": "source_1"}),
    Document(page_content="Texto do documento 2", metadata={"source": "source_2"}),
]

### * Criação de Vector Store

In [87]:
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(all_splits, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Realiza a busca
query = "Quais são os conteúdos básicos de língua portuguesa?"
retrieved_docs = retriever.get_relevant_documents(query)

# Exibe os resultados
for idx, doc in enumerate(retrieved_docs):
    print(f"Documento {idx+1}:\n{doc.page_content[:500]}\n")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### * Retrieval

In [73]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [74]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### * Geração de Respostas

In [76]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [77]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

### Exemplo de uso:

In [90]:
query = input('Digite uma pergunta sobre o edital concurso do BNDES: ')
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Digite uma pergunta sobre o edital concurso do BNDES: Qual a data de inscrição?


NameError: name 'rag_chain' is not defined